# Limpieza de Datos

Carga de librerias y datos

In [139]:
import pandas as pd
import numpy as np
pd.set_option("display.max_colwidth", None)

In [140]:
# Carga de datos csv
Inmuebles = pd.read_csv(r'C:\Users\sebit\OneDrive\Documentos\Portafolio\Analisis Casas Valparaiso\Scraping icasas\Casas.csv')

### Corrección Datos numericos, separadores miles y decimales.

In [141]:
# Mucha informacion que nos interesa de forma numerica esta como object
Numerico = ['Precio UF','Area m2','Área útil','m2 terreno','Año de construcción']
for col in Numerico:
    Inmuebles[col] = (
        Inmuebles[col]
        .astype(str)
        .str.replace(".", "", regex=False)   # miles
        .str.replace(",", ".", regex=False)  # decimales
        .str.strip()
    )
for col in Numerico:
    Inmuebles[col] = pd.to_numeric(Inmuebles[col], errors="coerce")

In [142]:
# Revision 
print(Inmuebles.info())
Inmuebles.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 995 entries, 0 to 994
Data columns (total 49 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Precio UF                  995 non-null    float64
 1   N Habitaciones             974 non-null    float64
 2   Area m2                    927 non-null    float64
 3   N Baños                    967 non-null    float64
 4   Comuna                     995 non-null    object 
 5   Provincia                  995 non-null    object 
 6   Amueblado                  522 non-null    object 
 7   Gastos Comunes             147 non-null    object 
 8   Área útil                  181 non-null    float64
 9   Estacionamiento            546 non-null    object 
 10  m2 terreno                 929 non-null    float64
 11  Enlace                     995 non-null    object 
 12  Año de construcción        401 non-null    float64
 13  Agua                       237 non-null    object 

,Precio UF,N Habitaciones,Area m2,N Baños,Comuna,Provincia,Amueblado,Gastos Comunes,Área útil,Estacionamiento,...,Seguridad 24 horas,Vista panorámica,Balcón,Pieza de Servicio,Acceso a discapacitados,Cancha de Tenis,Nivel,Sólo familias,Sauna,Ascensor
0,12800.00,3.0,139.0,4.0,Concón,Valparaíso,Sin amueblar,: true,139.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23000.00,4.0,450.0,5.0,Viña del Mar,Valparaíso,Sin amueblar,NaN,450.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8081.80,4.0,160.0,3.0,Viña del Mar,Valparaíso,Sin amueblar,NaN,160.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8100.00,3.0,160.0,3.0,San Felipe,San Felipe de Aconcagua,Semi-amueblado,NaN,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9567.24,4.0,194.0,2.0,Quilpué,Valparaíso,Sin amueblar,NaN,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Correccion UF publicado como 0.45 a 17800 UF segun descripcion.

In [143]:
Inmuebles.loc[Inmuebles['Precio UF']==0.45,'Precio UF'] = 17800

No se considera Valores 100000 por sesgo y muy poca informacion.

In [144]:
Inmuebles = Inmuebles[Inmuebles['Precio UF'] <= 50000]

### Arreglo Comunas

Columnas bajo 40 o igual de frecuencia es Otras

In [145]:
conteos = Inmuebles['Comuna'].value_counts()

comunas_raras = conteos[conteos <= 40].index

# 3. Reemplazamos esas comunas raras por 'Otras'
Inmuebles['Comuna'] = Inmuebles['Comuna'].replace(comunas_raras, 'Otras')

# Verificación
print(Inmuebles['Comuna'].value_counts())

Comuna
Otras            274
Viña del Mar     255
Concón           118
Puchuncaví        96
Zapallar          77
Quilpué           63
Villa Alemana     60
Valparaíso        45
Name: count, dtype: int64


#### Manejo Año de Construcción no coherente

* Se observo la columna de Año de Construcción tiene valores muy pequeños.
* Verificar con algunas paginas y confirmar que los valores estan siendo truncados
* **Solucion**: Los valores al ser truncados solo por 10**n agregar los 0 faltantes, cuando estos sean menores a 1000.

In [146]:
# Correccion de Año de construcción para valores entre 10 y 20.3
Condicion_10_20 = (Inmuebles['Año de construcción']>= 10) & (Inmuebles['Año de construcción'] <= 20.3)
Inmuebles.loc[Condicion_10_20, 'Año de construcción'] *= 100

# Correción de año de Año de construcción para valores entre 100 y 102.6 
Condicion_100_1026 = (Inmuebles['Año de construcción']>= 100) & (Inmuebles['Año de construcción'] <= 202.6)
Inmuebles.loc[Condicion_100_1026, 'Año de construcción'] *= 10

In [147]:
Inmuebles.head(20)

,Precio UF,N Habitaciones,Area m2,N Baños,Comuna,Provincia,Amueblado,Gastos Comunes,Área útil,Estacionamiento,...,Seguridad 24 horas,Vista panorámica,Balcón,Pieza de Servicio,Acceso a discapacitados,Cancha de Tenis,Nivel,Sólo familias,Sauna,Ascensor
0,12800.00,3.0,139.0,4.0,Concón,Valparaíso,Sin amueblar,: true,139.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23000.00,4.0,450.0,5.0,Viña del Mar,Valparaíso,Sin amueblar,NaN,450.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8081.80,4.0,160.0,3.0,Viña del Mar,Valparaíso,Sin amueblar,NaN,160.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8100.00,3.0,160.0,3.0,Otras,San Felipe de Aconcagua,Semi-amueblado,NaN,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9567.24,4.0,194.0,2.0,Quilpué,Valparaíso,Sin amueblar,NaN,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3144.60,3.0,90.0,2.0,Villa Alemana,Valparaíso,Sin amueblar,NaN,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10300.00,4.0,200.0,3.0,Quilpué,Valparaíso,NaN,NaN,200.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2341.46,NaN,840.0,NaN,Otras,Marga Marga,NaN,NaN,840.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2341.46,3.0,70.0,2.0,Quilpué,Valparaíso,NaN,NaN,70.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4750.00,3.0,84.0,2.0,Villa Alemana,Valparaíso,Sin amueblar,NaN,NaN,True,...,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [148]:
# Comprobación 
Comprobracion1 = Inmuebles.loc[Inmuebles['Año de construcción']<1700,['Enlace','Año de construcción']]
print(Comprobracion1) 

Comprobracion2 = Inmuebles.loc[Inmuebles['Año de construcción']> 2026]
print(Comprobracion2['Año de construcción']) 

                                                                  Enlace  \
64   https://www.icasas.cl/propiedad/77aa-80fc-198df32-c59439fb0277-725f   
589  https://www.icasas.cl/propiedad/af08-a449-1960305-92acdbf8af3f-7675   

     Año de construcción  
64                1500.0  
589                 21.0  
Series([], Name: Año de construcción, dtype: float64)


In [149]:
# NaN para ambos datos. (Segun publicadores son del año 1500 y 2100)
Inmuebles.loc[(Inmuebles['Año de construcción'] == 1500) | (Inmuebles['Año de construcción'] == 21),'Año de construcción'] = np.nan

In [150]:
# Comprobación 
Comprobracion1 = Inmuebles.loc[Inmuebles['Año de construcción']<1700,['Enlace','Año de construcción']]
print(Comprobracion1) 

Comprobracion2 = Inmuebles.loc[Inmuebles['Año de construcción']> 2026]
print(Comprobracion2['Año de construcción']) 

Empty DataFrame
Columns: [Enlace, Año de construcción]
Index: []
Series([], Name: Año de construcción, dtype: float64)


## Normalizar columnas de area en Area Construida y Area Terreno

In [151]:
Areas = ['Area m2', 'Área útil', 'm2 terreno']

Inmuebles['Area Construida'] = Inmuebles[Areas].max(axis=1)
Inmuebles['Area Terreno'] = Inmuebles[Areas].min(axis=1)

cols_originales = ['Area m2', 'Área útil', 'm2 terreno']
Inmuebles.drop(columns=cols_originales, inplace=True)

Inmuebles[['Area Construida','Area Terreno']].head()


,Area Construida,Area Terreno
0,300.0,139.0
1,1701.0,450.0
2,247.9,160.0
3,413.0,160.0
4,440.0,194.0


In [152]:
# Debido a la logica anterior es necesario dejar en NaN area construida si Area Terreno es igual a Area Construida.
Areas_iguales = (Inmuebles['Area Terreno']==Inmuebles['Area Construida'])
Inmuebles.loc[Areas_iguales, 'Area Construida']= np.nan

### Flag Terreno y Casa

* En caso de no tener Baños ni Habitaciones es Terreno, de lo contrario es casa.

In [153]:
# Se crean las columnas correspondientes.
Inmuebles['Casa'] = True
Inmuebles['Terreno'] = False

# En caso de no tener habitaciones ni baños es terreno
Terreno0 = (Inmuebles['N Habitaciones'] == 0) & (Inmuebles['N Baños'] == 0)
Terreno_NaN =  (Inmuebles['N Habitaciones'].isna()) & (Inmuebles['N Baños'].isna())
Inmuebles.loc[Terreno0 | Terreno_NaN, 'Terreno'] = True
Inmuebles.loc[Terreno0 | Terreno_NaN, 'Casa'] = False

# Rellenar valores faltantes N Habitaciones y N Baños cuando ambos son NaN
Hab_Baños = Inmuebles['N Habitaciones'].isna() & Inmuebles['N Baños'].isna()

# Rellenar con 0 
Inmuebles.loc[Hab_Baños, 'N Habitaciones'] = 0
Inmuebles.loc[Hab_Baños, 'N Baños'] = 0

# En caso de no cumplirse estas condiciones (Falta uno de los 2) Rellenar con mediana.
Inmuebles['N Habitaciones'] = Inmuebles['N Habitaciones'].fillna(Inmuebles['N Habitaciones'].median())
Inmuebles['N Baños'] = Inmuebles['N Baños'].fillna(Inmuebles['N Baños'].median())

### Eliminar duplicados

In [154]:
Caracteristicas = ['Precio UF', 'N Habitaciones', 'N Baños', 'Area Terreno', 'Comuna']
Inmuebles= Inmuebles.drop_duplicates(subset= Caracteristicas, keep= 'first').reset_index(drop=True)

### Eliminación servicios basicos.

In [155]:
Columnas=['Electricidad', 'Agua', 'Gas Natural', 'Estado de conservación', 'Nivel', 'Gastos Comunes', 'Sólo familias', 'Enlace']
Inmuebles = Inmuebles.drop(columns=Columnas)

### Trabajar como 0 y 1 para caracteristicas True y False

In [156]:
Columnas_Mantencion = ['Precio UF', 'N Habitaciones', 'N Baños', 'Comuna','Provincia', 'Amueblado', 'Enlace', 'Año de construcción','Area Construida', 'Area Terreno']

columnas_a_limpiar = [col for col in Inmuebles.columns if col not in Columnas_Mantencion]


# Aplicar la limpieza solo a las columnas restantes

# Rellenar vacíos con False
Inmuebles[columnas_a_limpiar] = Inmuebles[columnas_a_limpiar].fillna(False)

# Convertir a Enteros (1 y 0)
Inmuebles[columnas_a_limpiar] = Inmuebles[columnas_a_limpiar].astype(int)

print("Columnas mantenidas intactas:", Columnas_Mantencion)
print("Columnas limpiadas (0 y 1):", columnas_a_limpiar)

Columnas mantenidas intactas: ['Precio UF', 'N Habitaciones', 'N Baños', 'Comuna', 'Provincia', 'Amueblado', 'Enlace', 'Año de construcción', 'Area Construida', 'Area Terreno']
Columnas limpiadas (0 y 1): ['Estacionamiento', 'Aire Acondicionado', 'Alarma', 'Bodega', 'Chimenea', 'Placares', 'Cocina Equipada', 'Terraza', 'Portero', 'Centros comerciales', 'Cerca de Escuelas', 'Cocina Integrada', 'Internet', 'Patio', 'Aseos', 'Piscina', 'Quincho', 'Cerca de estación de tren', 'Calefacción', 'Jacuzzi', 'Jardín', 'Área juegos infantiles', 'Gimnasio', 'Seguridad 24 horas', 'Vista panorámica', 'Balcón', 'Pieza de Servicio', 'Acceso a discapacitados', 'Cancha de Tenis', 'Sauna', 'Ascensor', 'Casa', 'Terreno']


C:\Users\sebit\AppData\Local\Temp\ipykernel_13032\1656796039.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Inmuebles[columnas_a_limpiar] = Inmuebles[columnas_a_limpiar].fillna(False)


In [157]:
# Generación csv
Inmuebles.to_csv(r'C:\Users\sebit\OneDrive\Documentos\Portafolio\Analisis Casas Valparaiso\Datos ML Valparaiso.csv', index=False, encoding='utf-8-sig')